In [1]:
!nvidia-smi

Sat Jan  7 17:06:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   51C    P0   171W / 300W |  22233MiB / 32510MiB |     99%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [3]:
import sys
import os
import shutil
sys.path.append("../../")
sys.path.append("../")
sys.path.append("../../../../../")

In [4]:
# load important packages
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision import transforms
from trailmet.algorithms.binarize.utils import *
from trailmet.datasets.classification import DatasetFactory
from datasets import DataManager

In [5]:
from trailmet.algorithms.binarize.birealnet import BirealNet
from trailmet.models.mobilenet import make_mobilenetv2
# from trailmet.models.mobilenet_v2 import get_mobilenet

In [6]:
import yaml
root = './'
import os
with open(os.path.join(root,"birealnet_cifar100.yaml"),'r') as stream:
    kwargs = yaml.safe_load(stream)

In [7]:
kwargs

{'dataset': 'c100',
 'BACKBONE': 'mobilenetv2',
 'batch_size': 256,
 'epochs': 600,
 'optimizer': 'torch.optim.Adam',
 'lr': 0.0005,
 'momentum': 0.9,
 'weight_decay': 0,
 'save_path': 'save_path_mbnet_c100_0fp',
 'data_path': '',
 'label_smooth': 0.1,
 'device': 'cuda',
 'num_fp': 0,
 'workers': 6,
 'valid_size': 0.1,
 'num_train': 0,
 'num_classes': 100,
 'insize': 32}

In [8]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE')
    
set_seed(1024)  

> SEEDING DONE


In [9]:
crop_scale = 0.08
lighting_param = 0.1
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32, scale=(crop_scale, 1.0)),
    Lighting(lighting_param),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()])

valid_transform = transforms.Compose([
        transforms.Resize(36),
        transforms.CenterCrop(32),
        transforms.ToTensor()
    ])
data_object = DataManager(kwargs)
trainloader, valloader, testloader = data_object.prepare_data(train_transform, valid_transform)
dataloaders = {
        'train': trainloader, 'val': valloader, "test": testloader
}
# from torchvision.datasets import CIFAR100
# from torch.utils.data import DataLoader
# train_set = CIFAR100('./test_cifar', train=True, transform=train_transform, download=True)
# test_set = CIFAR100('./test_cifar', train=False, transform=valid_transform, download=True)
# train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=4, pin_memory=True)
# val_loader = DataLoader(test_set, batch_size=256, shuffle=False, num_workers=4, pin_memory=True)
# dataloaders = {
#         'train': train_loader, 'val': val_loader, "test": val_loader
# }

... Preparing data ...
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
using fixed split
45000 5000


In [10]:
model = make_mobilenetv2(num_classes=kwargs['num_classes'], num_fp=kwargs['num_fp'])
# model = make_birealnet50(num_classes=kwargs['num_classes'],insize=kwargs['insize'], num_fp=kwargs['num_fp'])

In [11]:
algo = BirealNet(model, dataloaders, **kwargs)

In [ ]:
algo.binarize()

01/07 05:06:24 PM CFG = {'dataset': 'c100', 'BACKBONE': 'mobilenetv2', 'batch_size': 256, 'epochs': 600, 'optimizer': 'torch.optim.Adam', 'lr': 0.0005, 'momentum': 0.9, 'weight_decay': 0, 'save_path': 'save_path_mbnet_c100_0fp', 'data_path': '', 'label_smooth': 0.1, 'device': 'cuda', 'num_fp': 0, 'workers': 6, 'valid_size': 0.1, 'num_train': 0, 'num_classes': 100, 'insize': 32}
01/07 05:06:24 PM MobileNetV2(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): HardBinaryConv()
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): BinaryActivation()
        (3): HardBinaryConv()
        (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

01/07 05:06:31 PM loading checkpoint save_path_mbnet_c100_0fp/c100-checkpoint.pth.tar ..........
01/07 05:06:32 PM loaded checkpoint save_path_mbnet_c100_0fp/c100-checkpoint.pth.tar epoch = 255
01/07 05:06:32 PM epoch, train accuracy, train loss, val accuracy, val loss
learning_rate: 0.0002866666666666666


/opt/conda/envs/ml/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch: [255][  0/176]	Time  3.045 ( 3.045)	Data  1.177 ( 1.177)	Loss 3.7408e+00 (3.7408e+00)	Acc@1  15.62 ( 15.62)	Acc@5  40.23 ( 40.23)
Epoch: [255][  1/176]	Time  0.223 ( 1.634)	Data  0.002 ( 0.590)	Loss 3.8366e+00 (3.7887e+00)	Acc@1  13.28 ( 14.45)	Acc@5  36.33 ( 38.28)
Epoch: [255][  2/176]	Time  0.176 ( 1.148)	Data  0.001 ( 0.394)	Loss 3.8738e+00 (3.8171e+00)	Acc@1  11.33 ( 13.41)	Acc@5  34.77 ( 37.11)
Epoch: [255][  3/176]	Time  0.148 ( 0.898)	Data  0.002 ( 0.296)	Loss 3.7825e+00 (3.8084e+00)	Acc@1  15.62 ( 13.96)	Acc@5  37.50 ( 37.21)
Epoch: [255][  4/176]	Time  0.191 ( 0.757)	Data  0.003 ( 0.237)	Loss 3.8042e+00 (3.8076e+00)	Acc@1  14.84 ( 14.14)	Acc@5  42.19 ( 38.20)
Epoch: [255][  5/176]	Time  0.163 ( 0.658)	Data  0.001 ( 0.198)	Loss 3.8814e+00 (3.8199e+00)	Acc@1  14.06 ( 14.13)	Acc@5  31.64 ( 37.11)
Epoch: [255][  6/176]	Time  0.123 ( 0.581)	Data  0.001 ( 0.170)	Loss 3.7949e+00 (3.8163e+00)	Acc@1  16.41 ( 14.45)	Acc@5  40.23 ( 37.56)
Epoch: [255][  7/176]	Time  0.134 ( 0.525

Epoch: [255][ 60/176]	Time  0.220 ( 0.199)	Data  0.003 ( 0.021)	Loss 3.7069e+00 (3.7457e+00)	Acc@1  20.31 ( 14.99)	Acc@5  44.14 ( 40.20)
Epoch: [255][ 61/176]	Time  0.174 ( 0.199)	Data  0.003 ( 0.021)	Loss 3.6290e+00 (3.7438e+00)	Acc@1  19.14 ( 15.06)	Acc@5  47.66 ( 40.32)
Epoch: [255][ 62/176]	Time  0.183 ( 0.199)	Data  0.003 ( 0.021)	Loss 3.6944e+00 (3.7430e+00)	Acc@1  15.23 ( 15.06)	Acc@5  41.80 ( 40.35)
Epoch: [255][ 63/176]	Time  0.119 ( 0.198)	Data  0.001 ( 0.020)	Loss 3.5973e+00 (3.7407e+00)	Acc@1  21.88 ( 15.17)	Acc@5  47.66 ( 40.46)
Epoch: [255][ 64/176]	Time  0.142 ( 0.197)	Data  0.001 ( 0.020)	Loss 3.6905e+00 (3.7399e+00)	Acc@1  14.84 ( 15.16)	Acc@5  44.92 ( 40.53)
Epoch: [255][ 65/176]	Time  0.128 ( 0.196)	Data  0.002 ( 0.020)	Loss 3.6971e+00 (3.7393e+00)	Acc@1  16.80 ( 15.19)	Acc@5  38.67 ( 40.50)
Epoch: [255][ 66/176]	Time  0.127 ( 0.195)	Data  0.004 ( 0.020)	Loss 3.6803e+00 (3.7384e+00)	Acc@1  16.41 ( 15.21)	Acc@5  44.92 ( 40.57)
Epoch: [255][ 67/176]	Time  0.182 ( 0.194

In [16]:
chk=torch.load('./save_path_mbnet_c100_0fp/c100-model_best.pth.tar')

In [17]:
chk['epoch']

597

In [18]:
chk['best_top1_acc']

tensor(28.7000)